### FILE ORGANISATION
<pre>
My project _____ _____ gpx ___ ___ trip1 ___ ___ 1.gpx
                |             |             |___ 2.gpx
                |             |             |___ 3.gpx
                |             |             |___ n.gpx
                |             |             |___ D+.txt (optional)*
                |             |
                |             |___ trip2 ___ ___ 1.gpx
                |                           |___ 2.gpx
                |                           |___ 3.gpx
                |                           |___ n.gpx
                |                           |___ D+.txt (optional)*
                |
                |
                |_____ Compil_gpx.ipynb
                |_____ index.html (will be generated when you will run the code)

* the D+.txt should be:
col0 = gpx name ("1", "2", "3", ... is used here)
col1 = D+ (units = m)
col2 = km
col3 = D+/km
</pre>

### PACKAGES

In [29]:
import folium
import gpxpy
import glob
import pandas as pd
import matplotlib.colors as colors
import os
import math
from folium import IFrame, Element

### GLOBAL PARAMETERS

In [30]:
center_lat = -50     # Move the initial map North-South (increase --> go towards North)
center_lon = -70    # Move the initial map Est-West (increase --> go towards Est)
zoom_start = 6      # Change the zoom level of the initial map (7 is more zoomed than 6)

### TRACKS NAME, COLOR, LEGEND...

In [31]:
voyages = {
    "South_America": {                     # Update
        "folder": "gpx",     # Update
        "type": "palette",
        "colors": ["#FFD000", "#FF0000"],           # Update

        "text": "Trip: Bordeaux-Angers <br>"      # Update
                "Date: November 2025 <br>"              # Update
                "Distance: ?? km <br>"                  # Update
                "Total D+: ?? m <br>"                   # Update
                "Duration: ?? <br>"                 # Update
                "Travelers: Charlotte LEU and Yann GAUCHER",              # Update

        "marker_position": (47.2184, -1.5536),          # Update (you can choose some city coordinates in "ADD CITIES" chapter)
    }
# # ------------------------------------------------------------------
#     "2025_Montpellier-Munich": {
#         "folder": "gpx/2025_Montpellier-Munich",
#         "type": "palette",
#         "colors": ["#FFD000", "#FF0000"],

#         "text": "Trip: Montpellier-Munich <br>"
#                 "Date: July-August 2025 <br>"
#                 "Distance: 2200 km <br>"              
#                 "Total D+: 16000 m <br>"
#                 "Duration: 2 months <br>"                         
#                 "Travelers: Billie LIBRI & Hugo REYMOND",

#         "marker_position": (43.7696, 11.2558),
#     },
# # ------------------------------------------------------------------
#     "2024_Alsace-Paris": {
#         "folder": "gpx/2024_Strasbourg-Paris",
#         "type": "fixed",
#         "color": "#FFA500",

#         "text": "Trip: Strasbourg-Paris <br>"
#                 "Date: April 2024 <br>"
#                 "Distance: 629 km <br>"              
#                 "Total D+: 6070 m <br>"
#                 "Duration: 5 days <br>"
#                 "Travelers: Clément GENOT & Hugo REYMOND",

#         "marker_position": (48.0727, 6.8990),
#     },
# # ------------------------------------------------------------------
#     "2022_Orléans-Bourgogne": {
#         "folder": "gpx/2022_Orleans-Bourgogne",
#         "type": "fixed",
#         "color": "#FFA500",

#         "text": "Trip: Orléans-Bourgogne <br>"
#                 "Date: July 2022 <br>"
#                 "Distance: 345 km <br>"               
#                 "Total D+: 3000 m <br>"
#                 "Duration: 5 days <br>"
#                 "Travelers: Billie LIBRI & Hugo REYMOND",

#         "marker_position": (46.9896, 3.1590),
#     }
}

### CREATE THE MAP

In [32]:
# You don't have to change anything here :-)

m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_start, tiles=None)
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr='Tiles © Esri',
    name="ESRI Satellite",
    overlay=False,
    control=True
).add_to(m)

### DRAW GPX

In [33]:
for voyage_name, params in voyages.items():
    folder = params["folder"]
    gpx_files = glob.glob(os.path.join(folder, "*.gpx"))

    if params["type"] == "palette":
        txt_path = os.path.join(folder, "D+.txt")
        if not os.path.exists(txt_path):
            print(f"Missing D+.txt for {voyage_name}, using default color")
            df = None
        else:
            df = pd.read_csv(txt_path, sep="\t", header=None)
            df = df.rename(columns={0: "gpx", 3: "Difficulty"})
            df["gpx"] = df["gpx"].astype(int)

        cmap = colors.LinearSegmentedColormap.from_list("gradient", params["colors"])
        if df is not None and not df.empty:
            norm = colors.Normalize(vmin=df["Difficulty"].min(), vmax=df["Difficulty"].max())
            get_color = lambda val: colors.to_hex(cmap(norm(val)))
        else:
            get_color = lambda val: "#FFD000"

        for gpx_file in gpx_files:
            filename = os.path.basename(gpx_file)
            try:
                gpx_num = int(os.path.splitext(filename)[0])
                if df is not None:
                    difficulte = df.loc[df["gpx"] == gpx_num, "Difficulty"].values[0]
                    color = get_color(difficulte)
                else:
                    color = "#FFD000"
            except Exception:
                color = "#FFD000"

            with open(gpx_file, 'r', encoding='utf-8') as f:
                gpx = gpxpy.parse(f)
                for track in gpx.tracks:
                    for segment in track.segments:
                        points = [(p.latitude, p.longitude) for p in segment.points]
                        if points:
                            folium.PolyLine(points, color=color, weight=4, opacity=0.9).add_to(m)

    elif params["type"] == "fixed":
        color = params["color"]
        for gpx_file in gpx_files:
            with open(gpx_file, 'r', encoding='utf-8') as f:
                gpx = gpxpy.parse(f)
                for track in gpx.tracks:
                    for segment in track.segments:
                        points = [(p.latitude, p.longitude) for p in segment.points]
                        if points:
                            folium.PolyLine(points, color=color, weight=4, opacity=0.9).add_to(m)

# --- MARKERS: fake info-sign pin with hover animation and pointer embedded ---
for voyage_name, params in voyages.items():
    pos = params["marker_position"]
    text = params.get("text", voyage_name)

    # Custom HTML for popup
    popup_html = f"""
    <div class="custom-popup">
        {text}
    </div>
    """
    popup = folium.Popup(
        folium.IFrame(popup_html, width=250, height=120),
        max_width=300,

    )

    # Custom HTML icon: circle + pointer embedded
    html_icon = f"""
    <div class="custom-info-pin" title="Click for trip details">
        <i class="fa fa-info-circle"></i>
        <div class="pin-point"></div>
    </div>
    """

    folium.Marker(
        location=pos,
        popup=popup,
        tooltip="ℹ️ Click for trip details",
        icon=folium.DivIcon(
            html=html_icon,
            icon_size=(32, 42),   # total size including pointer
            icon_anchor=(16, 42)  # bottom center
        )
    ).add_to(m)

# --- Add CSS for style, hover animation, and pointer embedded ---
hover_style = """
<style>
.custom-info-pin {
    background-color: #00ccff; /* Base blue */
    color: white;
    border-radius: 50%;
    width: 32px;
    height: 32px;
    display: flex;
    align-items: center;
    justify-content: center;
    box-shadow: 0 0 6px rgba(0, 204, 255, 0.8);
    transition: transform 0.25s ease, box-shadow 0.25s ease;
    position: relative;
    cursor: pointer;
}

.custom-info-pin:hover {
    transform: scale(1.3);
    box-shadow: 0 0 15px rgba(0, 204, 255, 1);
}

.custom-info-pin i {
    font-size: 18px;
}

.pin-point {
    width: 0;
    height: 0;
    border-left: 8px solid transparent;
    border-right: 8px solid transparent;
    border-top: 12px solid #00ccff; /* color same as circle */
    position: absolute;
    bottom: -8px;      /* embedded into the circle */
    left: 50%;
    transform: translateX(-50%);
}

/* --- Styled popup --- */
.custom-popup {
    background-color: rgba(0, 204, 255, 0.9); /* semi-transparent blue */
    color: white;
    padding: 10px 12px;
    border-radius: 8px;
    font-size: 14px;
    line-height: 1.3;
    box-shadow: 2px 2px 8px rgba(0,0,0,0.3);
}
</style>
"""

m.get_root().html.add_child(Element(hover_style))


### ADD CITIES

In [34]:
villes = {
"Punta Arenas": (-53.1638, -70.9171),
"Buenos Aires": (-34.6037, -58.3816),
"Santiago": (-33.4489, -70.6693),
"São Paulo": (-23.5505, -46.6333),
"Rio de Janeiro": (-22.9068, -43.1729),
"Brasília": (-15.8267, -47.9218),
"Bogotá": (4.7110, -74.0721),
"Lima": (-12.0464, -77.0428),
"Quito": (-0.1807, -78.4678),
"Caracas": (10.4806, -66.9036),
"Montevideo": (-34.9011, -56.1645),
"Asunción": (-25.2637, -57.5759),
"La Paz": (-16.4897, -68.1193),
"Sucre": (-19.0196, -65.2619),
"Valparaíso": (-33.0472, -71.6127),
"Córdoba": (-31.4201, -64.1888),
"Rosario": (-32.9587, -60.6930),
"Medellín": (6.2442, -75.5812),
"Guayaquil": (-2.1700, -79.9224),
"Barranquilla": (10.9685, -74.7813),
"Maracaibo": (10.6545, -71.6500),
}

# Compute offsets to avoid overlapping labels
proximity_thresh = 0.5
base_offset = 8
max_offset = 10
zoom_factor = 1.2
def haversine(lat1, lon1, lat2, lon2):
    return math.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)

offsets = {nom: (base_offset, -base_offset) for nom in villes.keys()}
noms = list(villes.keys())
for i in range(len(noms)):
    for j in range(i+1, len(noms)):
        city1, city2 = noms[i], noms[j]
        lat1, lon1 = villes[city1]
        lat2, lon2 = villes[city2]
        dist = haversine(lat1, lon1, lat2, lon2)
        if dist < proximity_thresh:
            offset_val = min(max_offset, (proximity_thresh - dist) / proximity_thresh * max_offset)
            dx = offset_val * zoom_factor
            dy = offset_val * zoom_factor
            offsets[city1] = (-dx, -dy)
            offsets[city2] = (dx, dy)

# Add cities to the map
fg_villes = folium.FeatureGroup(name="Cities")
for nom, coord in villes.items():
    folium.CircleMarker(
        location=coord,
        radius=3,
        color='white',
        fill=True,
        fill_color='green',
        fill_opacity=0.9
    ).add_to(fg_villes)
    dx, dy = offsets[nom]
    folium.map.Marker(
        location=coord,
        icon=folium.DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f"""<div style="font-size:14px; color:white; font-weight:bold;
                        text-shadow:1px 1px 2px black;
                        transform: translate({dx}px, {dy}px);">{nom}</div>"""
        )
    ).add_to(fg_villes)
fg_villes.add_to(m)

### SAVE THE MAP LOCALLY

In [35]:
# Layer control
folium.LayerControl().add_to(m)

# Save the map
output_file = "index.html"
m.save(output_file)
print(f"✅ Map generated: {output_file}")

✅ Map generated: index.html
